**IMPORT PACKAGES**

In [3]:
import requests

## Esta librería no te servirá en este análisis, pero
## es la librería básica para scrapear páginas más sencillas

from bs4 import BeautifulSoup as bs

## Esta sí. Con ella, sacaremos datos del HTML.

import random

## Para usar números aleatorios. Esta no era tan complicada.
## La usaremos para no usar el mismo tiempo siempre al scrapear.

import time

## Con esta metemos el retardo a la máquina. Las máquinas van rápidas. Yo no.

import pandas as pd
import numpy as np

## EL bread and butter del Data Science. Tablas y operaciones aritméticas.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## Esta es la manera para usar selenium normal, pero nosotros tenemos que parecer humanos.
## Por eso, usamos la siguiente librería.

import undetected_chromedriver as uc

## De persoon die dit heeft ontwikkeld is een crack. Alle proxy's geconfigureerd en
## anderen (ik zeg anderen omdat ik niet weet welke hekserij hij heeft gebruikt) om niet op te sporen te zijn.
## We zullen deze browser gebruiken.
import keyboard

from datetime import date

import playsound
import pygame
import datetime
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# CODE

### ALL PROPERTIES --> GENERAL INFORMATION

In [4]:
browser = uc.Chrome()

In [6]:
base = "https://www.idealista.com"
url = "https://www.idealista.com/en/venta-viviendas/barcelona-provincia/"
order = "?ordenado-por=fecha-publicacion-asc"
print(url)

https://www.idealista.com/en/venta-viviendas/barcelona-provincia/


In [7]:
browser.get(url)

## SET-UP:

### REGIONS BARCELONA IN IDEALISTA

In [4]:
regions = {}
regions['eixample']= ['la-dreta-de-l-eixample',
  'la-nova-esquerra-de-l-eixample',
  'l-antiga-esquerra-de-l-eixample',
  'la-sagrada-familia',
  'el-fort-pienc',
                     'sant-antoni']
regions[
 'ciutat-vella']= ['el-raval',
  'el-gotic',
  'sant-pere-santa-caterina-i-la-ribera',
  'la-barceloneta']

regions['sant-marti']= ['diagonal-mar-i-el-front-maritim-del-poblenou',
  'el-camp-de-l-arpa-del-clot',
  'el-besos',
  'el-clot',
  'la-verneda-i-la-pau',
  'el-poblenou',
  'la-vila-olimpica-del-poblenou',
  'sant-marti-de-provencals',
  'provencals-del-poblenou',
  'el-parc-i-la-llacuna-del-poblenou']
regions['sants-montjuic']= ['el-poble-sec-parc-de-montjuic',
  'sants',
  'la-marina-del-port',
  'sants-badal',
  'hostafrancs',
  'la-bordeta',
  'la-font-de-la-guatlla',
  'la-marina-del-prat-vermell',
  'zona-franca-port']
#regions = {}
regions["horta-guinardo"] = ["el-guinardo",
                            "el-carmel",
                            'el-baix-guinardo',
                            "la-teixonera",
                            "horta",
                            "can-baro",
                            "sant-genis-dels-agudells-montbau",
                             "la-vall-d-hebron-la-clota",
                            "la-font-d-en-fargues"]
regions["gracia"] = []
regions['les-corts'] = []
regions['nou-barris'] = []

regions['sarria-sant-gervasi'] = ['sant-gervasi-galvany',
                                  'el-putxet-i-el-farro',
                                  'sant-gervasi-la-bonanova',
                                  'sarria',
                                  'les-tres-torres',
                                  'vallvidrera-el-tibidabo-i-les-planes']
regions['sant-andreu'] = []

### REGIONS URL'S

In [5]:
for region in regions.keys():
    print (region)
    if len(regions[region])== 0:
        url = "https://www.idealista.com/en/venta-viviendas/barcelona/"+region+'/'
        #print(url)
    else:
        for sub_region in regions[region]:
            url = "https://www.idealista.com/en/venta-viviendas/barcelona/"+region+'/'+sub_region+'/'
            #print(url)

eixample
ciutat-vella
sant-marti
sants-montjuic
horta-guinardo
gracia
les-corts
nou-barris
sarria-sant-gervasi
sant-andreu


### WEBSCRAPER GENERAL INFO

**INPUT VARIABLES**

In [6]:
#url = "https://www.idealista.com/en/venta-viviendas/barcelona-barcelona/"
pygame.init()
alarm_sound = pygame.mixer.Sound("mixkit-classic-alarm-995.wav")
urls = []
df_webscrape_general = pd.DataFrame()
#save results on PC
save_webscrape = False
save_path_webscrape = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_General\''+"Barcelona_"


for region in regions.keys():
    print (region)
    page = 1
    stop = False
    if len(regions[region])== 0:
        url = "https://www.idealista.com/en/venta-viviendas/barcelona/"+region+'/'
    
        subregion = ''
        print(url)
    else:
        for sub_region in regions[region]:
            page = 1
            url = "https://www.idealista.com/en/venta-viviendas/barcelona/"+region+'/'+sub_region+'/'
            stop = False
            print(url)
        while stop == False:
            start = time.time()
            if page == 1:
                page_link = url+order
            else:
                pass
                page_link = url+"pagina-"+str(page)+'.htm'+order
            print(page_link)
            #scrape from pagelink
            browser.get(page_link)
            html = browser.page_source
            if len(html) < 5000:
                print ('Human Interaction Needed')
                print('Press Q to succeed, after puzzle is solved')
                stop = True
                while stop == True: 
                    alarm_sound.play()
                    if keyboard.is_pressed('q'):
                        stop = False
                html = browser.page_source

            #Beautiful soup
            soup = bs(html, 'lxml')

            #Scraping
            count = 0
            content = {}
            house_id = []
            house_number = []
            property_link = []
            title = []
            price = []
            main_detail = []
            description = []
            id = []
            main_content = soup.find('main',{'id':'main-content'})
            for item in main_content.find_all('div',{'class': 'item-info-container'}):
                house_number.append(count)
                main_info = item
                property_link.append(main_info.find("a",{'class': 'item-link'}, href=True)['href'])
                house_id.append(main_info.find("a",{'class': 'item-link'}, href=True)['href'].split("/")[-2])
                title.append(main_info.find('a',{'class':'item-link'})['title'])
                price.append(main_info.find('span', {'class': 'item-price h2-simulated'}).text.replace("€",''))
                main_detail.append(main_info.find('div',{'class':'item-detail-char'}).text.replace('\n',','))
                description.append(main_info.find('div',{'class':'item-description description'}).text.replace('\n',''))
                count += 1
            if count != 30:
                print(f'Strange no 30 items on {page_link}')

            #Content To Dictionary
            content['id'] = house_id
            content['house_number'] = house_number
            content['property_link'] = property_link
            content['title'] = title
            content['price'] = price
            content['main_detail'] = main_detail
            content['description'] = description

            #To excel
            df_webscrape = pd.DataFrame.from_dict(content)
            df_webscrape_general = pd.concat([df_webscrape_general, df_webscrape])
            today = date.today()
            
            title = save_path_webscrape+str(region)+'_'+str(sub_region)+'_'+str(today)+'_page_'+str(page)+'.csv'
            print(title)
            if save_webscrape:
                df_webscrape.to_csv(title)
            #Check if there is a next page in excistence  
            page += 1
            try: 
                web_next_link = base+ soup.find('div', {'class': 'pagination'}).find('li',{'class': 'next'}).find('a', href=True)['href']
            except:
                stop = True
            manual_page_link = url+"pagina-"+str(page)+'.htm'
            #print(web_next_link)
            #print(manual_page_link)
            main_content = soup.find('main',{'id':'main-content'})
            end = time.time()
            time_run = end-start
            time.sleep((np.random.randint(6,7)-time_run))

In [8]:
def general_scrape(url,region,sub_region, save_webscrape):
    page =1
    stop = False
    while stop == False:
        start = time.time()
        if page == 1:
            page_link = url+order
        else:
            pass
            page_link = url+"pagina-"+str(page)+'.htm'+order
        print(page_link)
        #scrape from pagelink
        browser.get(page_link)
        html = browser.page_source
        if len(html) < 5000:
            print ('Human Interaction Needed')
            print('Press Q to succeed, after puzzle is solved')
            stop = True
            while stop == True: 
                    alarm_sound.play()
                    if keyboard.is_pressed('q'):
                        stop = False
            html = browser.page_source

        #Beautiful soup
        soup = bs(html, 'lxml')
        
        #Scraping
        count = 0
        content = {}
        house_id = []
        house_number = []
        property_link = []
        title = []
        price = []
        main_detail = []
        description = []
        main_content = soup.find('main',{'id':'main-content'})
        for item in main_content.find_all('div',{'class': 'item-info-container'}):
            house_number.append(count)
            main_info = item
            property_link.append(main_info.find("a",{'class': 'item-link'}, href=True)['href'])
            house_id.append(main_info.find("a",{'class': 'item-link'}, href=True)['href'].split("/")[-2])
            title.append(main_info.find('a',{'class':'item-link'})['title'])
            price.append(main_info.find('span', {'class': 'item-price h2-simulated'}).text.replace("€",''))
            main_detail.append(main_info.find('div',{'class':'item-detail-char'}).text.replace('\n',','))
            description.append(main_info.find('div',{'class':'item-description description'}).text.replace('\n',''))
            count += 1
        if count != 30:
            print(f'Strange no 30 items on {page_link}')

        #Content To Dictionary
        content['id'] = house_id
        content['house_number'] = house_number
        content['property_link'] = property_link
        content['title'] = title
        content['price'] = price
        content['main_detail'] = main_detail
        content['description'] = description

        #To excel
        df_webscrape = pd.DataFrame.from_dict(content)
        today = date.today()
        title = save_path_webscrape+"Barcelona_"+str(region)+'_'+str(sub_region)+'_'+str(today)+'_page_'+str(page)+'.csv'
        #print(title)
        if save_webscrape:
            df_webscrape.to_csv(title)
        #Check if there is a next page in excistence  
        page += 1
        try: 
            web_next_link = base+ soup.find('div', {'class': 'pagination'}).find('li',{'class': 'next'}).find('a', href=True)['href']
        except:
            stop = True
        manual_page_link = url+"pagina-"+str(page)+'.htm'
        #print(web_next_link)
        #print(manual_page_link)
        main_content = soup.find('main',{'id':'main-content'})
        end = time.time()
        time_run = end-start
        #print(time_run)
        time.sleep((np.random.randint(7,10)))
    return

**WORKING WEBSCRAPE**

In [8]:
#url = "https://www.idealista.com/en/venta-viviendas/barcelona-barcelona/"
pygame.init()
alarm_sound = pygame.mixer.Sound("mixkit-classic-alarm-995.wav")
urls = []
for region in regions.keys():
    print (region)
    page = 1
    stop = False
    if len(regions[region])== 0:
        url = "https://www.idealista.com/en/venta-viviendas/barcelona/"+region+'/'
        sub_region = ''
        general_scrape(url,region,sub_region, False)
        print(f'Region finished: {region}.')
    else:
        for sub_region in regions[region]:
            page = 1
            url = "https://www.idealista.com/en/venta-viviendas/barcelona/"+region+'/'+sub_region+'/'
            stop = False
            #print(url)
            general_scrape(url,region,sub_region, False)
            print(f'Region {region}, subregion finished: {sub_region}')

eixample
https://www.idealista.com/en/venta-viviendas/barcelona/eixample/la-dreta-de-l-eixample/?ordenado-por=fecha-publicacion-asc
https://www.idealista.com/en/venta-viviendas/barcelona/eixample/la-dreta-de-l-eixample/pagina-2.htm?ordenado-por=fecha-publicacion-asc
https://www.idealista.com/en/venta-viviendas/barcelona/eixample/la-dreta-de-l-eixample/pagina-3.htm?ordenado-por=fecha-publicacion-asc
Human Interaction Needed
Press Q to succeed, after puzzle is solved


AttributeError: 'NoneType' object has no attribute 'find_all'

## WEBSCRAPING --> property specific informations

In [9]:
def collect_features(soup):
    features = {}
    try:
        features['title'] = soup.find('span',{'class':'main-info__title-main'}).text
    except:
        features['title'] = np.NaN
    try: 
        features['minor_title'] = soup.find('span',{'class':'main-info__title-minor'}).text
    except:
        features['minor_title'] = np.NaN
    try: 
        features['price'] = soup.find('span',{'class': 'info-data-price'}).text.replace("€",'')
    except:
        features['price'] = np.NaN
    try:
        features['m2'] =  soup.find('div', {'class': 'info-features'}).text.split('\n')[2]
        if features['m2'] == 'New home':
            features['m2'] =  soup.find('div', {'class': 'info-features'}).text.split('\n')[5]
    except:
        features['m2'] = np.NaN
    try:    
        basic_features = soup.find('div',{'class':'details-property-feature-one'}).find_all('ul')[0].text.split('\n')
        features['basic_features'] = ', '.join([x for x in basic_features if x != ""])
    except:
        features['basic_features'] = np.NaN
    try: 
        building_features = soup.find('div',{'class':'details-property-feature-one'}).find_all('ul')[1].text.split('\n')
        features['building_features'] = ', '.join([x for x in building_features if x != ""])
    except: 
        features['building_features'] = np.NaN
    try:
        features_two = soup.find('div',{'class':'details-property-feature-two'})
    except: 
        features_two = False
    if features_two != False:   
        amenities_info = False
        for word in soup.find('div',{'class':'details-property-feature-two'}).find('h3',{'class':'details-property-h3'}):
            #print(word)
            if 'Amenities' in word:
                amenities_info = True
            else:
                pass
        if amenities_info == True:
            amenities = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].text.split('\n')
            features['amenities'] = ', '.join([x for x in amenities if x != ""])
            energy_label = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].text.split('\n')
            energy_label = [x for x in energy_label if x != ""]
            #print(energy_label)
            try:
                features['energy_consumption'] = ', '.join([elem for elem in energy_label if 'kWh' in elem])
            except:
                features['energy_consumption'] = np.NaN
            try:
                features['energy_emmision'] = ', '.join([elem for elem in energy_label if 'CO2' in elem])
            except:
                features['energy_emmision'] = np.NaN
            try: 
                features['energy_label_consumption'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].find_all('span')[1]['title']
                if features['energy_label_consumption'] == "":
                        features['energy_label_consumption'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].find('span').text
            except: 
                features['energy_label_consumption'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].find('span').text
            try:   
                features['energy_label_emmision'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].find_all('span')[3]['title']
                if features['energy_label_emmision'] == "":
                        features['energy_label_emmision'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].find('span').text
            except:
                features['energy_label_emmision'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].find('span').text
        
        else:
            amenities = np.NaN 
            try:
                energy_label = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].text.split('\n')
                energy_label = [x for x in energy_label if x != ""]
                try:
                    features['energy_consumption'] = ', '.join([elem for elem in energy_label if 'kWh' in elem])
                except:
                    features['energy_consumption'] = np.NaN
                try:
                    features['energy_emmision'] = ', '.join([elem for elem in energy_label if 'CO2' in elem])
                except:
                    features['energy_emmision'] = np.NaN
                try: 
                    features['energy_label_consumption'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].find_all('span')[1]['title']
                    if features['energy_label_consumption'] == "":
                        features['energy_label_consumption'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].find('span').text
                except: 
                    features['energy_label_consumption'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].find('span').text
                try:
                    features['energy_label_emmision'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].find_all('span')[3]['title']
                    if features['energy_label_emmision'] == "":
                        features['energy_label_emmision'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].find('span').text
                except:

                    features['energy_label_emmision'] = soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[0].find('span').text
            except:
                pass
                
    try:
        location_info = soup.find('div', {'id': 'headerMap'}).text.split('\n')
        location_info = [x for x in location_info if x != ""]
    except:
        location_info = False
    if location_info != False:
        if len(location_info) == 5:
            features['street'] = 'Not Reported For Privacy'
            features['subdistrict'] = location_info[1]
            features['district'] = location_info[2]
            features['province'] = location_info[3]
            features['region_province'] = location_info[4]
        elif len(location_info) == 6:
            features['street'] = location_info[1]
            features['subdistrict'] = location_info[2]
            features['district'] = location_info[3]
            features['province'] = location_info[4]
            features['region_province'] = location_info[5]
        elif len(location_info) == 7:
            features['street'] = location_info[1]
            features['subdistrict'] = location_info[3]
            features['district'] = location_info[4]
            features['province'] = location_info[5]
            features['region_province'] = location_info[6]
                
        else:
            print(f'Length Location Info = {len(location_info)}.')
            print(location_info)
    try:

        features['price_per_m2'] = soup.find_all('span',{'class':'flex-feature-details'})[2].text.split('€')[0]
    except:
        features['price_per_m2'] = np.NaN
    try: 
        features['description'] = soup.find('div',{'class':"adCommentsLanguage expandable is-expandable"}).find('p').text.replace('\n','')
    except:
        features['description'] = np.NaN
    try:
        features['description'] = soup.find('div',{'class':"adCommentsLanguage expandable is-expandable with-expander-button"}).find('p').text.replace('\n','')
    except:
        pass
    try:
        script_tag = soup.find('script', text=re.compile('var mapConfig'))
        # Extract the text of the script tag containing the JavaScript code
        script_text = script_tag.text.strip()
        # Use regular expressions to extract the value of the mapConfig variable
        match = re.search(r'var mapConfig = ({.*});', script_text, re.DOTALL).group(1)
        latitude = match.split('\n')[1].replace('\t',"").split(',')[0].split(':')[1]
        longitude = match.split('\n')[2].replace('\t',"").split(',')[0].split(':')[1]
        latitude = re.findall(r'[\d,]+[\.]?\d*', latitude)
        features['latitude'] = float(latitude[0].replace(',', ''))
        longitude = re.findall(r'[\d,]+[\.]?\d*', longitude)
        features['longitude'] = float(longitude[0].replace(',', ''))

    except:
        features['latitude'] = np.NaN
        features['longitude'] = np.NaN
    try:
        features['n_photos'] = soup.find('span',{'class':'fa-button-text'}).text
    except:
        features['n_photos'] = np.NaN 
    try:
        info_features =  soup.find('div',{'class': 'info-features'}).text.split('\n')
        info_features = [x for x in info_features if x != ""]
        features['info_features'] = ', '.join(info_features)
    
        
    except:
        features['info_features']= np.NaN 
    try:
        features['last_update'] = soup.find('p',{'class': 'stats-text'}).text
    except:
        features['last_update'] = np.NaN
    try: 
        features['reference_listing'] = soup.find('p',{'class':'txt-ref'}).text.split('\n')[1]
    except:
        features['reference_listing'] = np.NaN
    try:
        features['prof_advertiser'] = soup.find('div',{'class': 'professional-name'}).text.split('\n')[5]
    except: 
        features['prof_advertiser'] = np.NaN
    try:
        features['private_advertiser'] = soup.find('div',{'class': 'professional-name'}).text.split('\n')[6]
    except:
        features['private_advertiser'] = np.NaN
    return features

**INPUT VARIABLES**

In [12]:
df = pd.read_csv('sample_18_04_2023.csv')
property_links = df.property_link

In [13]:
house_n = 0 #for breaks
last_house = house_n
page = 1
pygame.init()
alarm_sound = pygame.mixer.Sound("mixkit-classic-alarm-995.wav")
save_webscrape = False
save_path_webscrape = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_Specific\''

**RUNNING SCRAPER**

In [17]:
df_webscrape_specific = pd.DataFrame()
for house in property_links[house_n:]:
    start = time.time()
    page_link = base + house
    print(f'Pagelink: {page_link} for housenumber {house_n}')
    #Retrieve Code
    browser.get(page_link)
    html = browser.page_source
    if len(html) < 5000:
        print ('Human Interaction Needed')
        print('Press Q to succeed, after puzzle is solved')
        stop = True
        while stop == True:  
            alarm_sound.play()
            if keyboard.is_pressed('q'):
                stop = False
        html = browser.page_source
    #Beautiful soup
    soup = bs(html, 'lxml')
    try: 
        excist1 = soup.find('div',{'role': 'region'}).find('h1').text.replace("\n", "").replace("\t", "")
    except:
        excist1 = True
    try:
        excist2 = soup.find('div',{'class':'deactivated-detail_container'}).find('h1').text.replace("\n", "").replace("\t", "")
    except:
        excist2 = True
    "ERROR BECAUSE PAGE DOESN'T"
    if excist1 == "We couldn't find what you're looking for":
        print(f"Page: {page_link} does not excist anymore.")
    elif excist2 == "Sorry, this listing is no longer published":
        print(f"Page: {page_link} does not excist anymore.")
    else:
        features = collect_features(soup)
        features['page_link'] = page_link
        features['id'] = page_link.split("/")[-2]
        today = date.today()
        if house_n % 500 == 0:
            if page != 1:
                title = save_path_webscrape+"Barcelona_"+str(today)+"_house_"+str(last_house)+'_'+str(house_n)+'.csv'
                if save_webscrape:
                    df.to_csv(title)
                last_house = house_n
                #break
            else:
                pass
            page +=1
            df_webscrape_specific = pd.concat([df_webscrape_specific, df])
            df = pd.DataFrame()
        else:
            pass
        #print(features)
        df_dictionary = pd.DataFrame([features])
        df = pd.concat([df, df_dictionary], ignore_index=True)
        
        #df.append(features, ignore_index= True)
        #print(df)
        end = time.time()
        run_time = end-start 
        #if house_n == 505:
         #   break
        time.sleep((np.random.randint(5,8)))
        #time.sleep(run_time+5)
        house_n +=1
if save_webscrape:
    title = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_Specific\''+"Barcelona_"+str(today)+"_house_"+str(last_house)+'_'+str(house_n)+'.csv'
    df.to_csv(title) 

Pagelink: https://www.idealista.com/en/inmueble/31702050/ for housenumber 0
Human Interaction Needed
Press Q to succeed, after puzzle is solved


AttributeError: 'NoneType' object has no attribute 'find'

# GARBAGE AFTERWARDS?

# DON'T LOOK AT IT

In [11]:
title = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_Specific\''+"Barcelona_"+str(today)+"_house_"+str(last_house)+'_'+str(house_n)+'.csv'

In [13]:
title = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_Specific\''+"Barcelona_"+str(today)+"_house_"+str(last_house)+'_'+str(house_n)+'.csv'
df.to_csv(title) 

In [512]:
soup.find('div',{'class':'details-property-feature-two'}).find_all('ul')[1].find_all('span')[1]['title']

'b'

In [435]:
script_tag = soup.find('script', text=re.compile('var mapConfig'))
# Extract the text of the script tag containing the JavaScript code
script_text = script_tag.text.strip()
# Use regular expressions to extract the value of the mapConfig variable
match = re.search(r'var mapConfig = ({.*});', script_text, re.DOTALL).group(1)
latitude = match.split('\n')[1].replace('\t',"").split(',')[0].split(':')[1]
longitude = match.split('\n')[2].replace('\t',"").split(',')[0].split(':')[1]

In [447]:
latitude

" '41.3977814'"

In [448]:
result = re.findall(r'[\d,]+[\.]?\d*', latitude)
my_number = float(result[0].replace(',', ''))

In [449]:
my_number

41.3977814

In [239]:
date = pd.to_datetime('2023-04-13')
#date = date.strftime("%d_%m_%Y")


In [24]:
regions.keys()

dict_keys(['horta-guinardo', 'gracia', 'les-corts', 'nou-barris', 'sarria-sant-gervasi', 'sant-andreu'])

In [9]:
df_all = pd.DataFrame()
file = 0
for region in regions.keys():
    if len(regions[region])== 0:
        stop = False
        sub_region = ''
        page = 1
        while stop == False:
            try:
                day = datetime.date(2023,4,18)
                #base_adres = "C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_General"
                title = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_General\''+"Barcelona_"+str(region)+'_'+str(sub_region)+'_'+str(day)+'_page_'+str(page)+'.csv'
                df = pd.read_csv(title)
                #print(df)
                #df = pd.read_csv(r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_General\''+"Barcelona_"+str(region)+'_'+str(sub_region)+'_'+str(date)+'_page_'+str(page)+'.csv')
                page +=1
                file +=1
                df_all = pd.concat([df_all, df])
            except: 
                
                stop =True
    else:
        for sub_region in regions[region]:
            page = 1
            stop = False
            while stop == False:
                try:
                    day = datetime.date(2023,4,18)
                    title = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_General\''+"Barcelona_"+str(region)+'_'+str(sub_region)+'_'+str(day)+'_page_'+str(page)+'.csv'
                    df = pd.read_csv(title)  
                    df_all = pd.concat([df_all, df])
                    page +=1 
                    file +=1
                except:
                    stop = True
                    

In [27]:
file

572

**DROP DUPLICATES**

In [19]:
print(len(df_all))
df_filtered = df_all[~df_all.duplicated(subset='id', keep=False)]
print(len(df_filtered))

16455
16441


In [20]:
df_filtered.to_csv('sample_18_04_2023.csv')

In [8]:
df = pd.read_csv('sample_18_04_2023.csv')
property_links = df.property_link

**WEBSCRAPE**

In [261]:
date

datetime.date

In [22]:
house_n = 0
last_house = 0
page = 1
for house in property_links:
    start = time.time()
    page_link = base + house
    print(page_link)
    #Retrieve Code
    browser.get(page_link)
    html = browser.page_source
    if len(html) < 5000:
        print ('Human Interaction Needed')
        print('Press Q to succeed, after puzzle is solved')
        stop = True
        while stop == True:  
            if keyboard.is_pressed('q'):
                stop = False
        html = browser.page_source
    #Beautiful soup
    soup = bs(html, 'lxml')
    features = collect_features(soup)
    features['page_link'] = page_link
    features['id'] = page_link.split("/")[-2]
    today = date.today()
    if house_n % 1000 == 0:
        if page != 1:
            title = r'C:\Users\niels\python\UPF\Thesis\Barcelona_Housing_Specific\''+"Barcelona_"+str(today)+str(last_house)+'_'+str(house_n)+'.csv'
            df.to_csv(title)
            last_house = house_n
        else:
            pass
        page +=1
        df = pd.DataFrame()
    else:
        pass
    #print(features)
    df_dictionary = pd.DataFrame([features])
    df = pd.concat([df, df_dictionary], ignore_index=True)
    #df.append(features, ignore_index= True)
    #print(df)
    end = time.time()
    run_time = end-start   
    time.sleep((np.random.randint(run_time,run_time+3)))
    house_n +=1
    

https://www.idealista.com/en/inmueble/31702050/
Human Interaction Needed
Press Q to succeed, after puzzle is solved
https://www.idealista.com/en/inmueble/33148486/
https://www.idealista.com/en/inmueble/39158016/
https://www.idealista.com/en/inmueble/40351755/
https://www.idealista.com/en/inmueble/93970881/
https://www.idealista.com/en/inmueble/84114683/
https://www.idealista.com/en/inmueble/84144318/
https://www.idealista.com/en/inmueble/84809865/
https://www.idealista.com/en/inmueble/85156127/
https://www.idealista.com/en/inmueble/85475409/
https://www.idealista.com/en/inmueble/85723149/
https://www.idealista.com/en/inmueble/85887505/
https://www.idealista.com/en/inmueble/86073242/
https://www.idealista.com/en/inmueble/86943143/
https://www.idealista.com/en/inmueble/81848703/
https://www.idealista.com/en/inmueble/87224917/
https://www.idealista.com/en/inmueble/87455193/
https://www.idealista.com/en/inmueble/88324275/
https://www.idealista.com/en/inmueble/89252787/
https://www.idealist

AttributeError: 'NoneType' object has no attribute 'find'

In [243]:
date.today()

Timestamp('2023-04-13 10:02:59.867815')

# GARBAGE

In [613]:
features = collect_features(soup)
features

({'title': 'Flat / apartment for sale in Jules Verne',
  'minor_title': 'El Putxet i el Farró, Barcelona',
  'price': '250,000 ',
  'm2': '108',
  'basic_features': ['108 m² built',
   '1 bedroom',
   '2 bathrooms',
   'Second hand/good condition'],
  'building_features': ['Ground floor interior', 'With lift'],
  'energy_label_consumption': '120 kWh/m² year',
  'energy_label_emmision': '36 kg CO2/m² year',
  'street': 'JULES VERNE',
  'subdistrict': 'Subdistrict El Putxet i el Farró',
  'district': 'District Sarrià-Sant Gervasi',
  'province': 'Barcelona',
  'region_province': 'Barcelona region, Barcelona',
  'price_per_m2': '2,315 ',
  'description': '108m2 apartment in Jules Verne, next to General Miter. The apartment, on the ground floor, is divided into different spaces on three levels: low, upper level and lower level. It has access from the stairs.Without identity card but habitable or leaseable as a study.Singular distributionOn the ground floor is the entrance to the house, the

In [647]:
df

,title,minor_title,price,m2,basic_features,building_features,ammenities,street,subdistrict,district,...,description,latitude,longitude,n_photos,info_features,last_update,reference_listing,advertiser,page_link,id
0,Flat / apartment for sale in calle de Vidal i ...,"El Poblenou, Barcelona","1,900,000",152,"[152 m² built, 118 m² floor area, 2 bedrooms, ...","[14th floor exterior, With lift]",[Air conditioning],Calle de Vidal i de Valenciano,Subdistrict El Poblenou,District Sant Martí,...,"Spectacular 135m2 penthouse, located in front ...",'41.4010252','2.2093365',5 photos,None,Listing updated on 2 March,VL031,Nova urbe,https://www.idealista.com//en/inmueble/86203153/,86203153


In [502]:
script_tag

<script>
	var config = {
		locale: 'en',
		userAuthenticated: false,
		isAuthenticatedPrivate: false,
		
		propertyId: 86363047,
		favoriteFirstTime: true,
		discardFirstTime: true,
		urlAddFavorite: '/add-favorite.htm',
		urlAdNewContactInfo: '/en/ajax/listingController/adContactInfoForDetail.ajax',
		addFavoriteTargetId: '32',
		discardTargetId: '39',
		viewStatisticsTargetId: '40',
		urlRemoveFavorite: '/remove-favorite.htm',
		urlAddDiscarded: '/add-ruled-out-detail.htm',
		urlRemoveDiscarded: '/remove-ruled-out.htm',
		urlAddComment: '/update-favorite-comment.htm',
		urlContact: '/en/ajax/contact/2/sendcontact.ajax',
		urlCounterOffer: '/en/ajax/contact/suggested/4/sendcontact.ajax',
		urlContactDetailGallery: '/en/ajax/contact/42/sendcontact.ajax',
		urlContactDetailGalleryVirtualTour: '/en/ajax/contact/suggested/sendcontact.ajax',
		urlContactDetailGalleryFloorPlan: '/en/ajax/contact/45/sendcontact.ajax',
		urlContactDetailGallery3DTour: '/en/ajax/contact/46/sendcontact.ajax',
	

In [514]:
import json
script_tag = soup.find('script', text=re.compile('var mapConfig'))
# Extract the text of the script tag containing the JavaScript code
script_text = script_tag.text.strip()
# Use regular expressions to extract the value of the mapConfig variable
match = re.search(r'var mapConfig = ({.*});', script_text, re.DOTALL).group(1)
latitude = match.split('\n')[1]
longitude = match.split('\n')[2]
latitude.replace('\t','')
longitude.replace('\t','')
latitude = latitude[1]
longitude = longitude[1]

    

In [518]:
latitude.replace('\t','')

"latitude: '41.4029567',"

In [510]:
m2 =  soup.find('div', {'class': 'info-features'}).text.split('\n')[2]
m2

'108'

**INFORMATION FUNCTION**

In [ ]:
titulo = soup.find('span',{'class':'main-info__title-main'}).text

In [ ]:
localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]


In [ ]:
c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})

In [110]:
precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))

'<html lang="en" env="es" username="" data-userauth="false" class=""><head>\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Property for sale in Barcelona province, Spain: houses and flats — idealista</title>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<meta name="description" content="62,648 houses and flats for sale in Barcelona, Spain, from 5,000 euros. Property in Barcelona for sale direct from owners and real estate agents. idealista, the leading real estate marketplace in Spain.">\n<meta name="author" content="idealista.com">\n<meta http-equiv="cleartype" content="on">\n<meta name="pragma" content="no-cache">\n<meta http-equiv="Pragma" content="no-cache">\n<meta http-equiv="Expires" content="Fri, 01 Jan 1990 00:00:00 GMT">\n<meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate">\n<script async="" src="https://js.datadome.co/tags.js"></script><script type="text/javascript" async="" charset="utf-8" id="utag_2

In [ ]:
soup = bs(html, 'lxml')
soup

Good!

Now, let's go to a house that we want to scrape. I have selected this one, but put the one you prefer.

In [85]:
house = "100704643"

In [96]:
#url = "https://www.idealista.com/inmueble/97786099/"
url =  "https://www.idealista.com/inmueble/"+house+ "/"
#url ='https://www.habitaclia.com/'
print(url)

https://www.idealista.com/inmueble/100704643/



let's go to that house

In [97]:
browser.get(url)

Oops, you have to press the cookies! You can't click on any of the information until you click on it.

Don't worry, we look for the xpath as in the video and click on it.

(You can also click manually. Nobody will know)

In [49]:
browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="didomi-notice-agree-button"]"}
  (Session info: chrome=112.0.5615.50)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B2DCE3+50899]
	(No symbol) [0x00ABE111]
	(No symbol) [0x009C5588]
	(No symbol) [0x009F08F9]
	(No symbol) [0x009F0AFB]
	(No symbol) [0x00A1F902]
	(No symbol) [0x00A0B944]
	(No symbol) [0x00A1E01C]
	(No symbol) [0x00A0B6F6]
	(No symbol) [0x009E7708]
	(No symbol) [0x009E886D]
	GetHandleVerifier [0x00D93EAE+2566302]
	GetHandleVerifier [0x00DC92B1+2784417]
	GetHandleVerifier [0x00DC327C+2759788]
	GetHandleVerifier [0x00BC5740+672048]
	(No symbol) [0x00AC8872]
	(No symbol) [0x00AC41C8]
	(No symbol) [0x00AC42AB]
	(No symbol) [0x00AB71B7]
	BaseThreadInitThunk [0x772900F9+25]
	RtlGetAppContainerNamedObjectPath [0x77D47BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77D47B8E+238]



Now, let's get the html out of the page.

In [92]:
html = browser.page_source
html

'<html lang="es" env="es" username="" data-userauth="false" class=""><head>\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title>Piso en venta en FLOTA DE INDIAS s/n, Ramón de Carranza - Madre Rafols, Sevilla — idealista</title>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<meta name="description" content="piso de 222 m², Piso en venta en  FLOTA DE INDIAS s/n, Ramón de Carranza - Madre Rafols, Sevilla, barrio Ramón de Carranza - Madre Rafols">\n<meta name="author" content="idealista.com">\n<meta http-equiv="cleartype" content="on">\n<meta name="pragma" content="no-cache">\n<meta http-equiv="Pragma" content="no-cache">\n<meta http-equiv="Expires" content="Fri, 01 Jan 1990 00:00:00 GMT">\n<meta http-equiv="Cache-Control" content="no-cache, no-store, must-revalidate">\n<script async="" src="https://js.datadome.co/tags.js"></script><script type="text/javascript" async="" charset="utf-8" id="utag_239" src="https://tag.aticdn.net/piano-analytics.js

How ugly is it?

We are going to use BeautifulSoup to make it much better. And apt to get information.

In [93]:
soup = bs(html, 'lxml')
soup

<html class="" data-userauth="false" env="es" lang="es" username=""><head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Piso en venta en FLOTA DE INDIAS s/n, Ramón de Carranza - Madre Rafols, Sevilla — idealista</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="piso de 222 m², Piso en venta en  FLOTA DE INDIAS s/n, Ramón de Carranza - Madre Rafols, Sevilla, barrio Ramón de Carranza - Madre Rafols" name="description"/>
<meta content="idealista.com" name="author"/>
<meta content="on" http-equiv="cleartype"/>
<meta content="no-cache" name="pragma"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Fri, 01 Jan 1990 00:00:00 GMT" http-equiv="Expires"/>
<meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/>
<script async="" src="https://js.datadome.co/tags.js"></script><script async="" charset="utf-8" id="utag_239" src="https://tag.aticdn.net/piano-analytics.js" type="text/javascript"><


Perfect, now we can start to extract the data.

We do the same thing that appears in the video.

In [255]:
soup

<html class="" data-userauth="false" env="es" lang="en" username=""><head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Property for sale in Barcelona province, Spain: houses and flats — idealista</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="62,666 houses and flats for sale in Barcelona, Spain, from 5,000 euros. Property in Barcelona for sale direct from owners and real estate agents. idealista, the leading real estate marketplace in Spain." name="description"/>
<meta content="idealista.com" name="author"/>
<meta content="on" http-equiv="cleartype"/>
<meta content="no-cache" name="pragma"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Fri, 01 Jan 1990 00:00:00 GMT" http-equiv="Expires"/>
<meta content="no-cache, no-store, must-revalidate" http-equiv="Cache-Control"/>
<script async="" data-id="C5OI1OSVNBDLN9M57370" src="https://analytics.tiktok.com/i18n/pixel/static/main.MTZiZDQ3NTA5MA.js" type="te

In [246]:
main_content = soup.find('main',{'id':'main-content'})
for item in main_content.find_all('div',{'class': 'item-info-container'}):
    main_info = item
    property_link = main_info.find("a",{'class': 'item-link'}, href=True)['href']
    title = main_info.find('a',{'class':'item-link'})['title']
    price = main_info.find('span', {'class': 'item-price h2-simulated'}).text.replace("€",'').replace(',',".")
    main_detail = main_info.find('div',{'class':'item-detail-char'}).text.replace('\n',',')
    description = main_info.find('div',{'class':'item-description description'}).text.replace('\n','')

In [252]:
a = soup.find('main',{'id':'main-content'})


In [254]:
for item in a.find_all('div',{'class': 'item-info-container'}):
    print(item)
    print('JAAAAAAAAA')
#a.find_all('div',{'class': 'item-info-container'})

<div class="item-info-container">
<picture class="logo-branding">
<a data-click="1" data-markup="listado::logo-agencia" href="/en/pro/barcelona-sothebys-international-realty/" title="Barcelona &amp; Costa Brava Sotheby’s Realty">
<img alt="Barcelona &amp; Costa Brava Sotheby’s Realty" src="https://st3.idealista.com/60/8c/f3/barcelona-sothebys-international-realty.gif"/>
</a>
</picture>
<a aria-level="2" class="item-link" href="/en/inmueble/100770574/" role="heading" title="Flat in calle dels Mirallers, Sant Pere - Santa Caterina i la Ribera, Barcelona">
Flat in calle dels Mirallers, Sant Pere - Santa Caterina i la Ribera, Barcelona
</a>
<div class="price-row">
<span class="item-price h2-simulated">940,000<span class="txt-big">€</span></span>
</div>
<div class="item-detail-char">
<span class="item-detail">1 bed.</span>
<span class="item-detail">129 m²</span>
<span class="item-detail">1st floor exterior with lift</span>
</div>
<div class="item-description description">
<p class="ellipsis

In [27]:
localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]
localizacion

AttributeError: 'NoneType' object has no attribute 'text'

In [28]:
precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))
precio

AttributeError: 'NoneType' object has no attribute 'text'

In [65]:
c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})
c1

<div class="details-property-feature-one">
<h3 class="details-property-h3">Características básicas</h3>
<div class="details-property_features">
<ul>
<li>114 m² construidos</li>
<li>3 habitaciones</li>
<li>2 baños</li>
<li>Balcón</li>
<li>Plaza de garaje incluida en el precio</li>
<li>Segunda mano/buen estado</li>
<li>Trastero</li>
<li>Orientación norte, oeste</li>
</ul>
</div>
<h3 class="details-property-h3">Edificio</h3>
<div class="details-property_features">
<ul>
<li>Planta 10ª exterior</li>
<li>Con ascensor</li>
</ul>
</div>
</div>

In [66]:
caract_basicas = [caract.text.strip() for caract in c1.find_all('li')]
caract_basicas

['114 m² construidos',
 '3 habitaciones',
 '2 baños',
 'Balcón',
 'Plaza de garaje incluida en el precio',
 'Segunda mano/buen estado',
 'Trastero',
 'Orientación norte, oeste',
 'Planta 10ª exterior',
 'Con ascensor']

In [67]:
c2 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-two'})
c2

<div class="details-property-feature-two">
<h3 class="details-property-h3">Equipamiento</h3>
<div class="details-property_features">
<ul>
<li>
Aire acondicionado
</li>
</ul>
</div>
<h3 class="details-property-h3">Certificado energético</h3>
<div class="details-property_features">
<ul>
<li>
<span>Consumo: </span>
<span class="icon-energy-c-e" title="e"></span>
</li>
<li>
<span>Emisiones: </span>
<span class="icon-energy-c-e" title="E"></span>
</li>
<div class="energy-certificate-dropdown">
<a class="energy-certificate-open-link" data-click="1" data-markup="A|ceeLabelShow" href="">Ver etiqueta de calificación energética</a>
<a class="energy-certificate-close-link" data-click="1" data-markup="A|ceeLabelHide" href="">Ocultar etiqueta</a>
<figure class="energy-certificate-img-container">
<span class="energy-certificate-img-ticket-left left-e" data-value-left-cee=""></span>
<span class="energy-certificate-img-ticket-right right-e" data-value-right-cee=""></span>
<img alt="idealista CEE" clas

In [68]:
caract_extra = [caract.text.strip() for caract in c2.find_all('li')]
caract_extra

['Aire acondicionado', 'Consumo:', 'Emisiones:']

In [69]:
ubicacion = soup.find('div',{'id':'headerMap'})
ubicacion

<div class="clearfix" id="headerMap">
<h3 class="ide-box-detail-h3">Ubicación</h3>
<ul>
<li class="header-map-list">
Paseo de García Faria
</li>
<li class="header-map-list">
Barrio Diagonal Mar i el Front Marítim del Poblenou
</li>
<li class="header-map-list">
Distrito Sant Martí
</li>
<li class="header-map-list">
Barcelona
</li>
<li class="header-map-list">
Área de Barcelona, Barcelona
</li>
</ul>
</div>

In [70]:
ubicacion_desg = [zona.text.strip() for zona in ubicacion.find_all('li')]
ubicacion_desg

['Paseo de García Faria',
 'Barrio Diagonal Mar i el Front Marítim del Poblenou',
 'Distrito Sant Martí',
 'Barcelona',
 'Área de Barcelona, Barcelona']

Ok, we have already extracted all the data we want!

The good thing is that, if you want, you can get the data you want.

We're going to automate this for houses in your area.

In [73]:
url_grande = 'https://www.idealista.com/venta-viviendas/sevilla/los-remedios/'
#url_grande = 'https://www.idealista.com/venta-viviendas/barcelona-barcelona/'
print (url_grande)

https://www.idealista.com/venta-viviendas/sevilla/los-remedios/


Well yes, it's still my area. But put yours!

Now, we want to get the id of each house. Let's get the html of the big page:

In [78]:
browser.get(url_grande)

In [79]:
#browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()

In [80]:
html = browser.page_source

In [81]:
html

'<html><head><title>idealista.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style><meta name="viewport" content="width=device-width, initial-scale=1.0"></head><body style="margin:0"><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMAlX0ifl90ZsQAXz6_sA==\',\'hsh\':\'AC81AADC3279CA4C7B968B717FBB30\',\'t\':\'fe\',\'s\':17156,\'e\':\'1c3ceca84bf51b8e1f1604f5d3ea4c0a9f37e21a6b708ae089c6ebad5384be19\',\'host\':\'geo.captcha-delivery.com\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script><script>if("string"==typeof navigator.userAgent&&navigator.userAgent.indexOf("Firefox")>-1){var isIframeLoaded=!1,maxTimeoutMs=5e3;function iframeOnload(e){isIframeLoaded=!0;var a=document.getElementById("noiframe");a&&a.parentNode.removeChild(a)}var initialTime=(new Date).getTime();setTimeout(function(){isIframeLoaded||(new Date).getTime()-initialTime>maxTimeoutMs&&(document.body.innerHTML=

In [77]:
soup = bs(html, 'lxml')
soup

<html><head><title>idealista.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style><meta content="width=device-width, initial-scale=1.0" name="viewport"/></head><body style="margin:0"><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMAlX0ifl90ZsQAXz6_sA==','hsh':'AC81AADC3279CA4C7B968B717FBB30','t':'fe','s':17156,'e':'98b132ee3aa2644a4c51b0633d50d12ecda6c1687cfcf26cbf0c3c26416ade3b','host':'geo.captcha-delivery.com'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script><script>if("string"==typeof navigator.userAgent&&navigator.userAgent.indexOf("Firefox")>-1){var isIframeLoaded=!1,maxTimeoutMs=5e3;function iframeOnload(e){isIframeLoaded=!0;var a=document.getElementById("noiframe");a&&a.parentNode.removeChild(a)}var initialTime=(new Date).getTime();setTimeout(function(){isIframeLoaded||(new Date).getTime()-initialTime>maxTimeoutMs&&(document.body.innerHTML='<div id="noiframe">Please

Now we locate where each house is and list them:

In [ ]:
articles = soup.find('main',{'class':'listing-items'}).find_all('article')
articles

Seleccionemos los ids!

In [ ]:
id_muebles = [article.get('data-adid') for article in articles]
id_muebles

Filtramos los nones:

In [ ]:
id_muebles = [muebles for muebles in id_muebles if muebles is not None]
id_muebles

Now that we have the ids of **one**, we have to repeat this process for each page in our area. I explain this process very well in the video.

In [39]:
busqueda = 'los remedios'

busqueda = busqueda.replace(' ','-')

x=1
ids = []

while True:
    
    url = f'https://www.idealista.com/venta-viviendas/sevilla/{busqueda}/pagina-{x}.htm'
    
    browser.get(url)
    
    time.sleep(random.randint(15,20))
    print('sleep over')
    
    try:
        
        browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
    
    html = browser.page_source
    
    soup = bs(html, 'lxml')
    
    pagina_actual = int(soup.find('main',{'class':'listing-items'}).find('div',{'class':'pagination'}).find('li',{'class':'selected'}).text)
    
    if x == pagina_actual:

        articles = soup.find('main',{'class':'listing-items'}).find_all('article')

    else:
        break
    
    x = x+1
    
    for article in articles:
        
        id_muebles = article.get('data-adid')
        
        ids.append(id_muebles)
        
        time.sleep(random.randint(1,3))
        print(id_muebles)
    
    ids = [muebles for muebles in ids if muebles is not None]

sleep over


AttributeError: 'NoneType' object has no attribute 'find'

In [40]:
html

'<html><head><title>idealista.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style><meta name="viewport" content="width=device-width, initial-scale=1.0"></head><body style="margin:0"><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMAhPLWSK_pSf4AXz6_sA==\',\'hsh\':\'AC81AADC3279CA4C7B968B717FBB30\',\'t\':\'fe\',\'s\':17156,\'e\':\'46944fdbbeef6d3491a2102195c67b179c88fe632978a3e248932d7ab5b2c60e\',\'host\':\'geo.captcha-delivery.com\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script><script>if("string"==typeof navigator.userAgent&&navigator.userAgent.indexOf("Firefox")>-1){var isIframeLoaded=!1,maxTimeoutMs=5e3;function iframeOnload(e){isIframeLoaded=!0;var a=document.getElementById("noiframe");a&&a.parentNode.removeChild(a)}var initialTime=(new Date).getTime();setTimeout(function(){isIframeLoaded||(new Date).getTime()-initialTime>maxTimeoutMs&&(document.body.innerHTML=

In [ ]:
ids_casas = pd.DataFrame(ids)
ids_casas.columns = ['id']
ids_casas

Ole! We are going to save these ids so as not to lose them.

We'll save it to a csv.

Beware of the index= False trick!

In [ ]:
ids_casas.to_csv('ids_casas.csv',index = False)

Now we are going to use a technique widely used in programming: create something empty and fill it in.

In this case, we create a series called **houses** to which we are going to add the scraped data.


In [ ]:
casas = pd.Series()

Here's the function. I have left some prints commented, uncomment them if you want to see them while we scrape.

In [ ]:
def parsear_inmueble(id_inmueble):
    
    print( '\n Casa numero: ' + id_inmueble)
    
    url = "https://www.idealista.com/inmueble/" + id_inmueble + "/"
    
    browser.get(url)

    html = browser.page_source
    
    soup = bs(html, 'lxml')

    titulo = soup.find('span',{'class':'main-info__title-main'}).text
    
    print('\n Titulo: ' + titulo)
    
    localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]

    print('\n Localizacion: ' + localizacion)
    precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))

    c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})

    caract_basicas = [caract.text.strip() for caract in c1.find_all('li')]
    
    #print('Caracteristicas basicas:' + caract_basicas)

    c2 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-two'})

    caract_extra = [caract.text.strip() for caract in c2.find_all('li')]
    
    #print('Caracteristicas extras:' + caract_extra)
    
    casas['titulo'] = titulo
    
    casas['localizacion'] = localizacion
    
    casas['precio'] = precio
    
    casas['caracteristicas_basicas'] = caract_basicas
    
    casas['caracteristicas_extras'] = caract_extra
    
    df_casas = pd.DataFrame(casas)
    
    return(df_casas.T)

In [ ]:
df_casas = parsear_inmueble(ids_casas.iloc[0].id)

for i in range(1,len(ids)):
    
    df_casas = pd.concat([df_casas,parsear_inmueble(ids[i])])
    
    time.sleep(random.randint(4,8))

A ver si funciona...

In [ ]:
df_casas

Perfecto! reseteemos el índice y guardemos estos datos en otro csv.

In [ ]:
df_casas.reset_index(drop=True, inplace=True)

In [ ]:
df_casas

In [ ]:
df_casas.to_csv('casas_idealista.csv', index = False, sep = ';', encoding = 'utf-16')

## So far the web scraping module.

## If you liked it, share the video with friends and tell them how to get this notebook.

## If you need anything, email me: javidatascience@gmail.com. Regards, and thank you very much!!

In [49]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

# https://stackoverflow.com/questions/70485179/runtimeerror-when-using-undetected-chromedriver
driver = uc.Chrome(use_subprocess=True)

results = {}

def extract_provinces():
    driver.get("https://www.idealista.com/")
    print(driver.page_source)
    input("press any key to continue...")
    provinces_div = driver.find_element(By.CLASS_NAME, 'locations-list')
    provinces = provinces_div.find_elements(By.XPATH, './/a')
    for province in provinces:
        results[province.text] = {
            "url": province.get_attribute('href'),
            "municipalities": {}
        }

def extract_municipalities(province, url):
    driver.get(url)
    municipalities = driver.find_elements(By.XPATH, '//ul[@id="location_list"]//a')
    results[province]['municipalities'] = {}
    for municipality in municipalities:
        results[province]['municipalities'][municipality.text] = {
            "url": municipality.get_attribute('href'),
            "properties": [],
        }

def extract_properties(url):
    driver.get(url)
    property_divs = driver.find_elements(By.XPATH, "//article[contains(@class, 'item')]")
    properties = []
    for div in property_divs:
        property = {}
        property['title'] = div.find_element(By.XPATH, './/a[@class="item-link"]').text
        property['url'] = div.find_element(By.XPATH, './/a[@class="item-link"]').get_attribute('href')
        property['price'] = div.find_element(By.XPATH, './/div[contains(@class, "price-row")]').text
        property['detail'] = div.find_element(By.XPATH, './/div[@class="item-detail-char"]').text
        property['description'] = div.find_element(By.XPATH, './/div[contains(@class, "item-description")]').text
        properties.append(property)
        
    if driver.find_elements(By.CLASS_NAME, "next"):
        url = driver.find_element(By.XPATH, "//li[@class='next']/a").get_attribute("href")
        properties += extract_properties(url)
    return properties


if __name__ == "__main__":
    extract_provinces()
    for province in results.keys():
        extract_municipalities(province, results[province]['url'])
        for municipality in results[province]['municipalities'].keys():
            municipality_properties = extract_properties(results[province]['municipalities'][municipality]['url'])
            results[province]['municipalities'][municipality]['properties'] = municipality_properties
    print(results)
#Avoid get

<html><head><title>idealista.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style><meta name="viewport" content="width=device-width, initial-scale=1.0"></head><body style="margin:0"><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMAWo5Bh7pGnlEAXz6_sA==','hsh':'AC81AADC3279CA4C7B968B717FBB30','t':'fe','s':17156,'e':'aa0ea0f325565bbceb43511721fd4d2a5816e6a0e4f7bdb13a6d86514c38c976','host':'geo.captcha-delivery.com'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script><script>if("string"==typeof navigator.userAgent&&navigator.userAgent.indexOf("Firefox")>-1){var isIframeLoaded=!1,maxTimeoutMs=5e3;function iframeOnload(e){isIframeLoaded=!0;var a=document.getElementById("noiframe");a&&a.parentNode.removeChild(a)}var initialTime=(new Date).getTime();setTimeout(function(){isIframeLoaded||(new Date).getTime()-initialTime>maxTimeoutMs&&(document.body.innerHTML='<div id="noiframe">Please 

KeyboardInterrupt: Interrupted by user

In [53]:
html = driver.page_source
soup = bs(html, 'lxml')
soup

<html><head><title>idealista.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style><meta content="width=device-width, initial-scale=1.0" name="viewport"/></head><body style="margin:0"><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMAWo5Bh7pGnlEAXz6_sA==','hsh':'AC81AADC3279CA4C7B968B717FBB30','t':'fe','s':17156,'e':'aa0ea0f325565bbceb43511721fd4d2a5816e6a0e4f7bdb13a6d86514c38c976','host':'geo.captcha-delivery.com'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script><script>if("string"==typeof navigator.userAgent&&navigator.userAgent.indexOf("Firefox")>-1){var isIframeLoaded=!1,maxTimeoutMs=5e3;function iframeOnload(e){isIframeLoaded=!0;var a=document.getElementById("noiframe");a&&a.parentNode.removeChild(a)}var initialTime=(new Date).getTime();setTimeout(function(){isIframeLoaded||(new Date).getTime()-initialTime>maxTimeoutMs&&(document.body.innerHTML='<div id="noiframe">Please

SyntaxError: invalid syntax (3220295134.py, line 1)

In [52]:
driver.page_source

'<html><head><title>idealista.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style><meta name="viewport" content="width=device-width, initial-scale=1.0"></head><body style="margin:0"><script data-cfasync="false">var dd={\'rt\':\'c\',\'cid\':\'AHrlqAAAAAMAWo5Bh7pGnlEAXz6_sA==\',\'hsh\':\'AC81AADC3279CA4C7B968B717FBB30\',\'t\':\'fe\',\'s\':17156,\'e\':\'aa0ea0f325565bbceb43511721fd4d2a5816e6a0e4f7bdb13a6d86514c38c976\',\'host\':\'geo.captcha-delivery.com\'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script><script>if("string"==typeof navigator.userAgent&&navigator.userAgent.indexOf("Firefox")>-1){var isIframeLoaded=!1,maxTimeoutMs=5e3;function iframeOnload(e){isIframeLoaded=!0;var a=document.getElementById("noiframe");a&&a.parentNode.removeChild(a)}var initialTime=(new Date).getTime();setTimeout(function(){isIframeLoaded||(new Date).getTime()-initialTime>maxTimeoutMs&&(document.body.innerHTML=